In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing import Annotated
from typing_extensions import TypedDict
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# ===== Stateクラスの定義 =====
class State(TypedDict):
    messages: Annotated[list, add_messages]

# ===== グラフの構築 =====
def build_graph(model_name):
    # ソースコードを記述
    # 検索ツールの定義
    tool = TavilySearchResults(max_results=2)
    tools = [tool]

    # グラフのインスタンスを作成
    graph_builder = StateGraph(State)

    # 言語モデルの定義
    llm = ChatOpenAI(model_name=model_name)

    # ツール定義の紐づけ
    llm_with_tools = llm.bind_tools(tools)

    # チャットボットノードの作成
    def chatbot(state: State):
        return {"messages": [llm_with_tools.invoke(state["messages"])]}
    
    # グラフにチャットボットノードを追加
    graph_builder.add_node("chatbot", chatbot)

    # ツールノードの作成
    tool_node = ToolNode(tools)

    # グラフにツールノードを追加
    graph_builder.add_node("tools", tool_node)

    # 条件付きエッジの作成
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )

    graph_builder.add_edge("tools", "chatbot")

    # 開始ノードの指定
    graph_builder.set_entry_point("chatbot")

    # メモリセーバーの作成
    memory = MemorySaver()
    return graph_builder.compile(checkpointer=memory)

# ===== グラフ実行関数 =====
def stream_graph_updates(graph: StateGraph, user_input: str):
    # ソースコードを記述
    events = graph.stream(
        {"messages": [("user", user_input)]},
        {"configurable": {"thread_id": "1"}},
        stream_mode="values"
    )
    # 結果をストリーミングで得る
    for event in events:
        print(event["messages"][-1].content, flush=True)

# ===== メイン実行ロジック =====
# 環境変数の読み込み
load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini" 

# グラフの作成
# ソースコードを記述
graph = build_graph(MODEL_NAME)

# メインループ
# ソースコードを記述
while True:
    user_input = input("質問：")
    if user_input.strip() == "":
        print("ありがとうございました！")
        break
    stream_graph_updates(graph, user_input)

こんにちは！
こんにちは！今日はどのようなお手伝いができるでしょうか？
１たす２は？
1たす2は3です。何か他に質問がありますか？
台湾観光んついて検索結果を教えて

[{"url": "https://www.hankyu-travel.com/guide/taiwan/", "content": "国内旅行\n\n海外旅行\n\nお役立ち\n\n旅行関連サービス\n\n台湾観光におすすめの名所＆人気のスポットランキング台湾観光ガイド\n\n絶景にグルメ、ショッピング、温泉まで楽しめる台湾。台北のシンボルタワー、台北101でショッピングや眺望を楽しんだり、夜は寧夏夜市でB級グルメ三昧。足をのばして、九份の街歩きもおすすめです。そんな台湾の基本情報から観光情報まで詳しく紹介します。\n\n台湾観光マップ\n\n台湾のおすすめ観光スポット総合ランキング\n\n九フン\n\n『千と千尋の神隠し』の舞台として脚光を浴びた山間部にある小さな町。赤提灯が並ぶ風景を求め、大勢の観光客が訪れます。\n\n士林夜市\n\n台北市最大規模の観光夜市です。台湾のローカルフードを提供する飲食屋台などさまざまな屋台が軒を連ねます。\n\n日月潭\n\n周囲を山々に囲まれた台湾最大の淡水湖。台湾3大観光地の一つとして人気で、四季折々の美しい景観が楽しめます。\n\n台北101\n\n地上101階建て、高さ508mの超高層ビル。毎年元旦0時にはカウントダウン花火を実施しています。\n\n野柳 [...] 日本統治時代を経験し、日本語を理解する人も多い台湾。しかし日本にいる感覚で行動するのは禁物。台湾の習慣やマナーを知って、現地の人と接しましょう。\n\n台湾の現地情報ブログ\n\n「ちょっと週末海外行ってくるわ」が可能！台湾に0泊1日で行く方法とモデルコース！\n\n【台湾】高雄の穴場！五つ星ホテル内「觀海軒」で高雄の眺望と絶品飲茶を！\n\n台湾に降り立ったらまずコンビニに行って欲しい6つの理由\n\n【MSCベリッシマ】4泊5日の沖縄美ら海クルーズ\n\n台湾の現地情報ブログ記事一覧を見る\n\n台湾旅行を探す\n\n関連特集\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n台湾観光ガイド\n\n海外観光ガイド\n\n観光ガイドトップ\n\n海外観光ガイド\n\nアジア\n\